TODO: OBIETTIVI DEL PROGETTO

TODO: ARCHITETTURA E TECNOLOGIE UTILIZZATE

RISULTATI:

In [1]:
#Import Libraries
from IPython.core.display import display
from pyspark.shell import spark

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.0.0
      /_/

Using Python version 3.7.3 (default, Mar  6 2020 22:34:30)
SparkSession available as 'spark'.


In [ ]:
#Reading all the 10 years CSV
year2005 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2005_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2006 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2006_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2007 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2007_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2008 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2008_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2009 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2009_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2010 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2010_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2011 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2011_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2012 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2012_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2013 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2013_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2014 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2014_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())
year2015 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/dataset/2015_data.csv',
                           header=True,
                           inferSchema=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True).cache())

In [3]:
# Merging all 11 years data into dataframe
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

MergeData = unionAll(year2005,
                     year2006,
                     year2007,
                     year2008,
                     year2009,
                     year2010,
                     year2011,
                     year2012,
                     year2013,
                     year2014,
                     year2015)

In [4]:
#Count of Merged Data
#MergeData.count()

In [5]:
#Count of Merged Data
#MergeData.printSchema()

In [6]:
# Create SQL Table from data frame
MergeData.registerTempTable("mergedTable")

In [7]:
# 1 - Male vs female deaths by resident status
results_male_female_resident_status = spark.sql(
  """SELECT resident_status,
            sex,
            count(sex) AS sex_counter
     FROM mergedTable
     GROUP BY resident_status, sex
     ORDER BY resident_status, sex""")

results_male_female_resident_status.show()

+---------------+---+-----------+
|resident_status|sex|sex_counter|
+---------------+---+-----------+
|              1|  F|   11419032|
|              1|  M|   11030268|
|              2|  F|    2027162|
|              2|  M|    2288859|
|              3|  F|     398187|
|              3|  M|     506845|
|              4|  F|      16495|
|              4|  M|      33825|
+---------------+---+-----------+



In [8]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'], port= 9042)
session = cluster.connect('mykeyspace')











In [9]:
stmt = session.prepare("INSERT INTO results_male_female_resident_status (i, resident_status, sex, sex_counter) VALUES (?, ?, ?, ?)")

In [10]:
for i, item in results_male_female_resident_status.toPandas().iterrows():
    #values = '{},{},{}'.format(i, item['resident_status'], item['sex'])
    #session.execute("INSERT INTO results_male_female_resident_status (i, resident_status, sex) VALUES ({})".format(values))
    results = session.execute(stmt, [i, item['resident_status'], item['sex'], item['sex_counter']])
    













In [11]:
# 2 - Male vs female deaths by month of the year
results_male_deaths_month = spark.sql(
  """SELECT month_of_death,
            sex,
            count(sex)
     FROM mergedTable
     GROUP BY month_of_death, sex
     ORDER BY month_of_death, sex""")

results_male_deaths_month.show()

+--------------+---+----------+
|month_of_death|sex|count(sex)|
+--------------+---+----------+
|             1|  F|   1302416|
|             1|  M|   1269199|
|             2|  F|   1177388|
|             2|  M|   1147291|
|             3|  F|   1262972|
|             3|  M|   1231322|
|             4|  F|   1150376|
|             4|  M|   1145743|
|             5|  F|   1134538|
|             5|  M|   1144327|
|             6|  F|   1066368|
|             6|  M|   1087923|
|             7|  F|   1088399|
|             7|  M|   1116795|
|             8|  F|   1083513|
|             8|  M|   1107852|
|             9|  F|   1065444|
|             9|  M|   1081367|
|            10|  F|   1144472|
|            10|  M|   1147902|
+--------------+---+----------+
only showing top 20 rows



In [12]:
# Read Disease description CSV (ICD10)
icd10 = (spark.read.csv(path='/Users/alessio/Documents/Projects/usa-mortality-analysis/sources/codes/ICD10.csv',
                        header=True,
                        inferSchema=True,
                        ignoreLeadingWhiteSpace=True,
                        ignoreTrailingWhiteSpace=True).cache())

In [13]:
#Create SQL Table of disease description
icd10.registerTempTable("icd10")

In [14]:
# 3 - Top 20 diseases causing deaths for either sex
results_diseases_causing_deaths_sex = spark.sql(
  """SELECT m.sex,
            m.icd_code_10th_revision,
            i.description3 AS description,
            count(m.sex)
     FROM mergedTable m, icd10 i
     WHERE i.code3 = m.icd_code_10th_revision
     GROUP BY i.description3 ,m.icd_code_10th_revision, m.sex
     ORDER BY count(m.sex) DESC, m.sex
     LIMIT 20""")

results_diseases_causing_deaths_sex.show()

+---+----------------------+--------------------+----------+
|sex|icd_code_10th_revision|         description|count(sex)|
+---+----------------------+--------------------+----------+
|  M|                  I219|Acute myocardial ...|    765648|
|  F|                  G309| Alzheimer's disease|    626180|
|  F|                  I219|Acute myocardial ...|    622679|
|  F|                  J449|Other chronic obs...|    602655|
|  M|                  J449|Other chronic obs...|    534241|
|  M|                   C61|Malignant neoplas...|    311877|
|  F|                  J189|Pneumonia, unspec...|    276400|
|  M|                  G309| Alzheimer's disease|    270020|
|  M|                  J189|Pneumonia, unspec...|    236009|
|  M|                  C189|Malignant neoplas...|    228669|
|  F|                  C189|Malignant neoplas...|    223213|
|  M|                  C259|Malignant neoplas...|    203498|
|  F|                  C259|Malignant neoplas...|    199666|
|  F|                  A

In [15]:
# 4 - Method of Disposition
results_disposition = spark.sql(
  """SELECT current_data_year AS Year,
            CASE method_of_disposition
            WHEN 'C' THEN 'Cremation'
            WHEN 'B' THEN 'Burial'
            WHEN 'D'THEN 'Donation'
            WHEN 'E' THEN 'Entombment'
            WHEN 'O' THEN 'Other'
            WHEN 'R' THEN 'RemovedFromUSA'
            WHEN 'U' THEN 'Unknown'
            END AS MethodofDisposition,
            COUNT(*) AS Count
     FROM mergedTable
     GROUP BY 1, 2
     ORDER BY 1, 3""")

results_disposition.show()

+----+-------------------+-------+
|Year|MethodofDisposition|  Count|
+----+-------------------+-------+
|2005|              Other|   2199|
|2005|           Donation|   4795|
|2005|         Entombment|  21247|
|2005|     RemovedFromUSA|  31954|
|2005|          Cremation| 350018|
|2005|             Burial| 553202|
|2005|            Unknown|1489091|
|2006|              Other|   2252|
|2006|           Donation|   6883|
|2006|         Entombment|  23412|
|2006|     RemovedFromUSA|  40870|
|2006|          Cremation| 423282|
|2006|             Burial| 667169|
|2006|            Unknown|1266857|
|2007|              Other|   3119|
|2007|           Donation|   8719|
|2007|         Entombment|  26139|
|2007|     RemovedFromUSA|  41411|
|2007|          Cremation| 472220|
|2007|             Burial| 725666|
+----+-------------------+-------+
only showing top 20 rows



In [16]:
# 5 - Manner of death per month
results_deaths_month = spark.sql(
  """SELECT month_of_death AS Month,
            CASE manner_of_death
            WHEN '0' THEN 'Not Specified'
            WHEN '1' THEN 'Accident'
            WHEN '2' THEN 'Suicide'
            WHEN '3' THEN 'Homicide'
            WHEN '4' THEN 'Pending investigation'
            WHEN '5' THEN 'Could not be determine'
            WHEN '6' THEN 'Self-Inflicted'
            WHEN '7' THEN 'Natural'
            ELSE 'OTHER'
            END AS MannerOfDeath,
            COUNT(*) AS Count
     FROM mergedTable
     GROUP BY 1, 2
     ORDER BY 1,2""")

results_deaths_month.show()

Py4JJavaError: An error occurred while calling o157.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 16 in stage 39.0 failed 1 times, most recent failure: Lost task 16.0 in stage 39.0 (TID 1509, 192.168.31.215, executor driver): org.apache.spark.memory.SparkOutOfMemoryError: Unable to acquire 262144 bytes of memory, got 33051
	at org.apache.spark.memory.MemoryConsumer.throwOom(MemoryConsumer.java:157)
	at org.apache.spark.memory.MemoryConsumer.allocateArray(MemoryConsumer.java:97)
	at org.apache.spark.unsafe.map.BytesToBytesMap.allocate(BytesToBytesMap.java:794)
	at org.apache.spark.unsafe.map.BytesToBytesMap.<init>(BytesToBytesMap.java:200)
	at org.apache.spark.unsafe.map.BytesToBytesMap.<init>(BytesToBytesMap.java:207)
	at org.apache.spark.sql.execution.UnsafeFixedWidthAggregationMap.<init>(UnsafeFixedWidthAggregationMap.java:101)
	at org.apache.spark.sql.execution.aggregate.HashAggregateExec.createHashMap(HashAggregateExec.scala:434)
	at org.apac

In [ ]:
# 6 - Analysis about Alzheimer's death order by group age
results_alzheimer = spark.sql(
  """SELECT count(*) as TotalDeaths,
            CASE age_recode_12
            WHEN '10' THEN '75 - 84 years'
            WHEN '11' THEN '85 years and over'
            WHEN '12' THEN 'Age not stated'
            WHEN '01' THEN 'Under 1 year'
            WHEN '02' THEN '1 - 4 years'
            WHEN '03' THEN '5 - 14 years'
            WHEN '04' THEN '15 - 24 years'
            WHEN '05' THEN '25 - 34 years'
            WHEN '06' THEN '35 - 44 years'
            WHEN '07' THEN '45 - 54 years'
            WHEN '08' THEN '55 - 64 years'
            WHEN '09' THEN '65 - 74 years'
            END AS age
     FROM mergedTable
     WHERE 113_cause_recode = '052'
     GROUP BY age
     ORDER BY TotalDeaths
     DESC limit 10""")

results_alzheimer.show()

In [ ]:
# 7 - Count number of graduate people who deads with suicide manner
results_degree_suicides = spark.sql(
  """SELECT count(*) as TotalDeaths,
            CASE education_2003_revision
            WHEN '1' THEN '8th grade or less'
            WHEN '2' THEN '9 - 12th grade, no diploma'
            WHEN '3' THEN 'high school graduate or GED completed'
            WHEN '4' THEN 'some college credit, but no degree'
            WHEN '5' THEN 'Associate degree'
            WHEN '6' THEN 'Bachelor’s degree'
            WHEN '7' THEN 'Master’s degree'
            WHEN '8' THEN 'Doctorate or professional degree'
            WHEN '9' THEN 'Unknown'
            END AS studies
     FROM mergedTable
     WHERE manner_of_death = '2'
     GROUP BY studies
     ORDER BY TotalDeaths
     DESC limit 10""")

results_degree_suicides.show()

In [ ]:
#Columns of Merge Data
MergeData.columns

TODO: DATA PRE-PROCESSING

In [ ]:
# Dropping Columns
MergeData = MergeData.drop('record_condition_1',
                           'record_condition_2',
                           'record_condition_3',
                           'record_condition_4',
                           'record_condition_5',
                           'record_condition_6',
                           'record_condition_7',
                           'record_condition_8',
                           'record_condition_9',
                           'record_condition_10',
                           'record_condition_11',
                           'record_condition_12',
                           'record_condition_13',
                           'record_condition_14',
                           'record_condition_15',
                           'record_condition_16',
                           'record_condition_17',
                           'record_condition_18',
                           'record_condition_19',
                           'record_condition_20')

MergeData = MergeData.drop('113_cause_recode',
                           '130_infant_cause_recode',
                           '39_cause_recode',
                           'number_of_entity_axis_conditions',
                           'entity_condition_1',
                           'entity_condition_2',
                           'entity_condition_3',
                           'entity_condition_4',
                           'entity_condition_5',
                           'entity_condition_6',
                           'entity_condition_7',
                           'entity_condition_8',
                           'entity_condition_9',
                           'entity_condition_10',
                           'entity_condition_11',
                           'entity_condition_12',
                           'entity_condition_13',
                           'entity_condition_14',
                           'entity_condition_15',
                           'entity_condition_16',
                           'entity_condition_17',
                           'entity_condition_18',
                           'entity_condition_19')

MergeData = MergeData.drop('icd_code_10th_revision',
                           'age_recode_27',
                           'age_recode_12',
                           'detail_age',
                           'entity_condition_20',
                           'education_2003_revision',
                           'education_1989_revision')

MergeData = MergeData.filter((MergeData.method_of_disposition == 'B') | (MergeData.method_of_disposition == 'C' ))

MergeData = MergeData.drop('detail_age_type',
                           'age_substitution_flag',
                           'age_substitution_flag',
                           'infant_age_recode_22',
                           'day_of_week_of_death',
                           'current_data_year',
                           '358_cause_recode',
                           'number_of_record_axis_conditions',
                           'hispanic_origin',
                           'race_recode_5')

In [ ]:
#Columns of Merge Data
MergeData.columns

In [ ]:
categoricalColumns = ['resident_status',
                      'month_of_death',
                      'sex',
                      'age_recode_52',
                      'place_of_death_and_decedents_status',
                      'marital_status',
                      'injury_at_work',
                      'manner_of_death',
                      'autopsy',
                      'activity_code',
                      'place_of_injury_for_causes_w00_y34_except_y06_and_y07_',
                      'race',
                      'race_recode_3',
                      'hispanic_originrace_recode']

In [ ]:
#Null Imputations
MergeData = MergeData.fillna({'place_of_injury_for_causes_w00_y34_except_y06_and_y07_': 12})
MergeData = MergeData.fillna({'activity_code': 11})
MergeData = MergeData.fillna({'manner_of_death': 999})
MergeData = MergeData.fillna({'place_of_death_and_decedents_status': 999})

# End Data pre-processing

In [ ]:
# Merging all 11 years data into dataframe
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

JoinedData=unionAll(year2005,
                    year2006,
                    year2007,
                    year2008,
                    year2009,
                    year2010,
                    year2011,
                    year2012,
                    year2013,
                    year2014,
                    year2015)

display(JoinedData)

In [ ]:
#Replacing null values in manner_of_death
JoinData = JoinedData.fillna({'manner_of_death': 12})
JoinData = JoinedData.filter(JoinedData.manner_of_death == '2')

In [ ]:
#Creating joint table on new joined data which is modified
JoinData.registerTempTable("jointTable")

In [ ]:
# 8 - Total suicides committed 2005-2015
results_suicides = spark.sql(
  """SELECT current_data_year AS Year,
            CASE manner_of_death
            WHEN '2' THEN 'Suicide'
            END AS TotalSuicidesCommited,
            COUNT(*) AS Count
     FROM jointTable
     GROUP BY 1, 2
     ORDER BY 1, 3""")

results_suicides.show()

In [ ]:
# 9 - Result about deaths pending
results_deaths_pending = spark.sql(
    """SELECT icd.description3, count(*) as TotalDeaths
       FROM mergedTable mt
       JOIN icd10 icd ON icd.code3 = mt.icd_code_10th_revision
       WHERE mt.manner_of_death == '4'
       GROUP BY icd.description3
       ORDER BY COUNT(*) DESC, icd.description3"""
)
results_deaths_pending.show()

In [ ]:
# 10 - Deaths for days of the week
results_day_week = spark.sql(
    """SELECT count(*) as TotalDeaths,
              CASE day_of_week_of_death
              WHEN '1' THEN 'Sunday'
              WHEN '2' THEN 'Monday'
              WHEN '3' THEN 'Tuesday'
              WHEN '4' THEN 'Wednesday'
              WHEN '5' THEN 'Thursday'
              WHEN '6' THEN 'Friday'
              WHEN '7' THEN 'Saturday'
              WHEN '9' THEN 'Unknown' END AS day
      FROM mergedTable
      GROUP BY day
      ORDER BY TotalDeaths"""
)

results_day_week.show()

In [ ]:
# 11 - Total deaths by skin's color
results_deaths_skin_color = spark.sql(
    """SELECT count(*) as TotalDeaths,
            CASE race_recode_3
            WHEN '1' THEN 'White'
            WHEN '2' THEN 'Races other than b&w'
            WHEN '3' THEN 'Black' END AS race
       FROM mergedTable
       GROUP BY race_recode_3
       ORDER BY TotalDeaths"""
)

results_deaths_skin_color.show()
